# Deployment

This notebook deploy a model trained in pipeline, and monitor it.

## Update Model Package Approval Status

We can approve the model using the SageMaker Studio UI or programmatically as shown below.

In [12]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [31]:
pipeline_name = 'qa-pipeline-16323001711632300171'

In [32]:
import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    #        print('Executions for our pipeline...')
    #        print(pipeline_execution_status)
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/67ot3yw4s5kf',
  'PipelineExecutionDisplayName': 'nidome',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2021, 9, 22, 9, 27, 48, 609000, tzinfo=tzlocal())},
 {'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/vqiccs4mntb5',
  'PipelineExecutionDisplayName': 'execution-1632300253350',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2021, 9, 22, 8, 44, 13, 274000, tzinfo=tzlocal())}]


### List Execution Steps

In [33]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [34]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/67ot3yw4s5kf


In [35]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 980000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model/pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg'}},
                             'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 370000, tzinfo=tzlocal()),
                             'StepName': 'CreateQAModel',
                             'StepStatus': 'Succeeded'},
                            {'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 187000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model-package/qamodelpackagegroup/7'}},
                             'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 298000, tzinfo=tzlocal()),
                             'StepName': 'QARegisterModel',
                             'StepStatus': 'Succeeded'},
                    

### View Registered Mode and Update Model Approval Status

In [36]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "QARegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

arn:aws:sagemaker:us-east-1:093729152554:model-package/qamodelpackagegroup/7


In [37]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",  # Other options are Rejected and PendingManualApproval
)

### View Created Model

In [38]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateQAModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

model_name = model_arn.split("/")[-1]
print(model_name)

arn:aws:sagemaker:us-east-1:093729152554:model/pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg
pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg


## Create Model Endpoint from Model Registry

More details here: https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html

In [39]:
import time

timestamp = int(time.time())

model_from_registry_name = "qa-model-from-registry-{}".format(timestamp)
print("Model from registry name : {}".format(model_from_registry_name))

model_registry_package_container = {
    "ModelPackageName": model_package_arn,
}

Model from registry name : qa-model-from-registry-1632311060


In [40]:
from pprint import pprint

create_model_from_registry_respose = sm.create_model(
    ModelName=model_from_registry_name, ExecutionRoleArn=role, PrimaryContainer=model_registry_package_container
)
pprint(create_model_from_registry_respose)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:093729152554:model/qa-model-from-registry-1632311060',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '95',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Wed, 22 Sep 2021 11:45:19 GMT',
                                      'x-amzn-requestid': 'e2d80cf9-e1d5-4b6b-b6ec-ec89e0fdaa9e'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'e2d80cf9-e1d5-4b6b-b6ec-ec89e0fdaa9e',
                      'RetryAttempts': 0}}


In [41]:
model_from_registry_arn = create_model_from_registry_respose["ModelArn"]
model_from_registry_arn

'arn:aws:sagemaker:us-east-1:093729152554:model/qa-model-from-registry-1632311060'

In [42]:
endpoint_config_name = "qa-model-from-registry-epc-{}".format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

qa-model-from-registry-epc-1632311060


In [43]:
pipeline_endpoint_name = "qa-model-from-registry-ep-{}".format(timestamp)
print("EndpointName={}".format(pipeline_endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

EndpointName=qa-model-from-registry-ep-1632311060
arn:aws:sagemaker:us-east-1:093729152554:endpoint/qa-model-from-registry-ep-1632311060


In [44]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

In [45]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

CPU times: user 234 ms, sys: 19.5 ms, total: 253 ms
Wall time: 10min 1s


### List All Artifacts

In [46]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

{'StepName': 'Processing', 'StartTime': datetime.datetime(2021, 9, 22, 9, 27, 49, 711000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 32, 16, 867000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:processing-job/pipelines-67ot3yw4s5kf-processing-jtj2dbws96'}}}
pipelines-67ot3yw4s5kf-processing-jtj2dbws96


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...22-08-43-33-756/input/code/preprocess.py,Input,DataSet,ContributedTo,artifact
1,s3://sm-nlp-data/nlu/data/qa_raw.zip,Input,DataSet,ContributedTo,artifact
2,68331...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://sm-nlp-data/nlu/data/processed/,Output,DataSet,Produced,artifact


{'StepName': 'Train', 'StartTime': datetime.datetime(2021, 9, 22, 9, 32, 17, 256000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 32, 17, 630000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'CacheHitResult': {'SourcePipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16323001711632300171/execution/vqiccs4mntb5'}, 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:training-job/pipelines-vqiccs4mntb5-train-jdmj5liqyt'}}}
pipelines-vqiccs4mntb5-train-jdmj5liqyt


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://sm-nlp-data/nlu/data/processed/,Input,DataSet,ContributedTo,artifact
1,76310...onaws.com/pytorch-training:1.8.1-gpu-py3,Input,Image,ContributedTo,artifact
2,s3://...tb5-Train-JDmj5LiQyt/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'EvaluateModel', 'StartTime': datetime.datetime(2021, 9, 22, 9, 32, 18, 202000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 55, 910000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:processing-job/pipelines-67ot3yw4s5kf-evaluatemodel-d7wij9sxb2'}}}


...

None

{'StepName': 'QARegisterModel', 'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 298000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 187000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model-package/qamodelpackagegroup/7'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,qamodelpackagegroup-7-Approved-1632311055-aws-...,Input,Approval,ContributedTo,action
1,s3://...tb5-Train-JDmj5LiQyt/output/model.tar.gz,Input,Model,ContributedTo,artifact
2,76310...naws.com/pytorch-inference:1.8.1-gpu-py3,Input,Image,ContributedTo,artifact
3,qamodelpackagegroup-7-PendingManualApproval-16...,Input,Approval,ContributedTo,action
4,QAModelPackageGroup-1631002331-aws-model-packa...,Output,ModelGroup,AssociatedWith,context


{'StepName': 'CreateQAModel', 'StartTime': datetime.datetime(2021, 9, 22, 9, 37, 57, 370000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 9, 22, 9, 37, 58, 980000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:093729152554:model/pipelines-67ot3yw4s5kf-createqamodel-en4hwj8lbg'}}}


None

### Test the Deployed Model

CSVSerializer: [DOC](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html#sagemaker.serializers.CSVSerializer) </br>
JSONDeserializer: [DOC](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.JSONDeserializer)

In [73]:
import json
from sagemaker.pytorch.model import PyTorchPredictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = PyTorchPredictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
    serializer=CSVSerializer(),
    deserializer=JSONDeserializer(),
)

In [54]:
!touch processed/psuedo/seq.in

In [74]:
with open('processed/psuedo/seq.in', mode='r') as f:
    lines = f.read()
lines

'伊坂幸太郎写了哪些书\nONE PIECE総集编 THE FIRST LOG是谁写的\n高效管理Windows网络/Win32 Perl应用之道的作者是谁\nDave写了什么书\n洪荒之武道是谁写的\n玄黄真人写了哪些书\n风景景观工程体系化是谁的作品\n微知汇：万物简史的作者是谁\n孽阳的作者是谁\n茅月写了哪些书\n未来娱乐系统是谁写的\n小僧不敲木鱼有什么著作\n金装四大才子是谁的作品\n罗永贤导演了哪些电影\n为了你我愿意热爱整个世界是谁的作品\n郭虎导演了哪些电影\n为了你我愿意热爱整个世界是谁导演的\n灭罪师的导演是谁\n杨苗导演了哪些电视剧\n灭罪师是谁导演的\n五百导演了哪些电视剧\n穆念慈的丈夫是谁\n杨康的配偶是谁'

In [81]:
with open('processed/psuedo/seq.in', 'r') as f:
    lines = f.readlines()
    predicted = predictor.predict(lines)
predicted

{'text': [['伊'],
  ['坂'],
  ['幸'],
  ['太'],
  ['郎'],
  ['写'],
  ['了'],
  ['哪'],
  ['些'],
  ['书'],
  [],
  [],
  ['O'],
  ['N'],
  ['E'],
  [],
  ['P'],
  ['I'],
  ['E'],
  ['C'],
  ['E'],
  ['総'],
  ['集'],
  ['编'],
  [],
  ['T'],
  ['H'],
  ['E'],
  [],
  ['F'],
  ['I'],
  ['R'],
  ['S'],
  ['T'],
  [],
  ['L'],
  ['O'],
  ['G'],
  ['是'],
  ['谁'],
  ['写'],
  ['的'],
  [],
  [],
  ['高'],
  ['效'],
  ['管'],
  ['理'],
  ['W'],
  ['i'],
  ['n'],
  ['d'],
  ['o'],
  ['w'],
  ['s'],
  ['网'],
  ['络'],
  ['/'],
  ['W'],
  ['i'],
  ['n'],
  ['3'],
  ['2'],
  [],
  ['P'],
  ['e'],
  ['r'],
  ['l'],
  ['应'],
  ['用'],
  ['之'],
  ['道'],
  ['的'],
  ['作'],
  ['者'],
  ['是'],
  ['谁'],
  [],
  [],
  ['D'],
  ['a'],
  ['v'],
  ['e'],
  ['写'],
  ['了'],
  ['什'],
  ['么'],
  ['书'],
  [],
  [],
  ['洪'],
  ['荒'],
  ['之'],
  ['武'],
  ['道'],
  ['是'],
  ['谁'],
  ['写'],
  ['的'],
  [],
  [],
  ['玄'],
  ['黄'],
  ['真'],
  ['人'],
  ['写'],
  ['了'],
  ['哪'],
  ['些'],
  ['书'],
  [],
  [],
  ['风'],
  ['景'],
  ['景'],
  ['观'],

In [80]:
predicted['text']

[['伊'],
 ['坂'],
 ['幸'],
 ['太'],
 ['郎'],
 ['写'],
 ['了'],
 ['哪'],
 ['些'],
 ['书'],
 [],
 [],
 ['O'],
 ['N'],
 ['E'],
 [],
 ['P'],
 ['I'],
 ['E'],
 ['C'],
 ['E'],
 ['総'],
 ['集'],
 ['编'],
 [],
 ['T'],
 ['H'],
 ['E'],
 [],
 ['F'],
 ['I'],
 ['R'],
 ['S'],
 ['T'],
 [],
 ['L'],
 ['O'],
 ['G'],
 ['是'],
 ['谁'],
 ['写'],
 ['的'],
 [],
 [],
 ['高'],
 ['效'],
 ['管'],
 ['理'],
 ['W'],
 ['i'],
 ['n'],
 ['d'],
 ['o'],
 ['w'],
 ['s'],
 ['网'],
 ['络'],
 ['/'],
 ['W'],
 ['i'],
 ['n'],
 ['3'],
 ['2'],
 [],
 ['P'],
 ['e'],
 ['r'],
 ['l'],
 ['应'],
 ['用'],
 ['之'],
 ['道'],
 ['的'],
 ['作'],
 ['者'],
 ['是'],
 ['谁'],
 [],
 [],
 ['D'],
 ['a'],
 ['v'],
 ['e'],
 ['写'],
 ['了'],
 ['什'],
 ['么'],
 ['书'],
 [],
 [],
 ['洪'],
 ['荒'],
 ['之'],
 ['武'],
 ['道'],
 ['是'],
 ['谁'],
 ['写'],
 ['的'],
 [],
 [],
 ['玄'],
 ['黄'],
 ['真'],
 ['人'],
 ['写'],
 ['了'],
 ['哪'],
 ['些'],
 ['书'],
 [],
 [],
 ['风'],
 ['景'],
 ['景'],
 ['观'],
 ['工'],
 ['程'],
 ['体'],
 ['系'],
 ['化'],
 ['是'],
 ['谁'],
 ['的'],
 ['作'],
 ['品'],
 [],
 [],
 ['微'],
 ['知'],
 ['汇'],
 ['：'],
 

In [ ]:
from inference import model_fn

res = model_fn('output')

In [10]:
import torch
torch.__version__

'1.6.0'

In [11]:
!aws s3 cp s3://sm-nlp-data/nlu/outputs/pipelines-4de5fz1k29hv-Train-ctml6o7XVi/output/model.tar.gz output/model/

download: s3://sm-nlp-data/nlu/outputs/pipelines-4de5fz1k29hv-Train-ctml6o7XVi/output/model.tar.gz to output/model/model.tar.gz


In [19]:
!cp processed/intent_label.txt output/train_meta/
!cp processed/slot_label.txt output/train_meta/

# Monitor SageMaker endpoints

Reference:
- SageMaker Doc: [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html)
- [AWS Workshop: Model Monitor](https://sagemaker-immersionday.workshop.aws/lab4/monitoring.html)

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri='s3://sm-nlp-data/inference/'
)

use the S3Downloader utility to view and download the captured data in Amazon S3: